In [24]:
import cx_Oracle
import pandas as pd
from datetime import datetime

In [25]:
# Load CSV
df = pd.read_csv("scraped data.csv")

In [30]:
try:
    conn = cx_Oracle.connect("bank_reviews", "BankPass123", "localhost:1521/XEPDB1")
    cursor = conn.cursor()
except cx_Oracle.DatabaseError as e:
    print(f"Database connection error: {e}")
    exit()

In [27]:
# Dictionary to map bank name to bank_id
bank_ids = {}

In [28]:
unique_banks = df['bank'].dropna().unique()

In [31]:
for bank_name in unique_banks:
    if pd.isna(bank_name) or bank_name.strip() == "":
        continue  # Skip invalid bank names
    try:
        # Assuming website and location are in CSV (add default values if not)
        website = str(df.loc[df['bank'] == bank_name, 'website'].iloc[0]) if 'website' in df.columns and not pd.isna(df.loc[df['bank'] == bank_name, 'website'].iloc[0]) else 'N/A'
        location = str(df.loc[df['bank'] == bank_name, 'location'].iloc[0]) if 'location' in df.columns and not pd.isna(df.loc[df['bank'] == bank_name, 'location'].iloc[0]) else 'N/A'
        cursor.execute("SELECT bank_id FROM banks WHERE name = :1", [bank_name])
        result = cursor.fetchone()
        if result:
            bank_ids[bank_name] = result[0]
        else:
            cursor.execute("INSERT INTO banks (name, website, location) VALUES (:1, :2, :3)", [bank_name, website, location])
            conn.commit()
            cursor.execute("SELECT bank_id FROM banks WHERE name = :1", [bank_name])
            bank_ids[bank_name] = cursor.fetchone()[0]
    except cx_Oracle.DatabaseError as e:
        print(f"Error inserting bank '{bank_name}': {e}")

Error inserting bank 'CBE': ORA-00942: table or view does not exist
Error inserting bank 'BOA': ORA-00942: table or view does not exist
Error inserting bank 'DB': ORA-00942: table or view does not exist


In [32]:
for index, row in df.iterrows():
    try:
        bank_name = row['bank']
        if pd.isna(bank_name) or bank_name.strip() == "":
            print(f"Skipping row {index}: Invalid bank name")
            continue
        bank_id = bank_ids.get(bank_name)
        if not bank_id:
            print(f"Skipping row {index}: Bank '{bank_name}' not found in bank_ids")
            continue

        # Handle CLOB for review_text
        review_text = str(row['review']) if not pd.isna(row['review']) else ""
        reviewer_name = str(row['reviewer_name']) if 'reviewer_name' in df.columns and not pd.isna(row['reviewer_name']) else "Anonymous"
        rating = int(row['rating']) if not pd.isna(row['rating']) else None
        review_date = pd.to_datetime(row['date'], errors='coerce')
        if pd.isna(review_date):
            review_date = datetime.now()

        # Use cursor.var() for CLOB
        review_text_clob = cursor.var(cx_Oracle.CLOB)
        review_text_clob.setvalue(0, review_text)

        cursor.execute("""
            INSERT INTO reviews (bank_id, reviewer_name, review_text, rating, review_date)
            VALUES (:1, :2, :3, :4, :5)
        """, (bank_id, reviewer_name, review_text_clob, rating, review_date))
    except Exception as e:
        print(f"Error inserting review at row {index}: {e}")

Skipping row 0: Bank 'CBE' not found in bank_ids
Skipping row 1: Bank 'CBE' not found in bank_ids
Skipping row 2: Bank 'CBE' not found in bank_ids
Skipping row 3: Bank 'CBE' not found in bank_ids
Skipping row 4: Bank 'CBE' not found in bank_ids
Skipping row 5: Bank 'CBE' not found in bank_ids
Skipping row 6: Bank 'CBE' not found in bank_ids
Skipping row 7: Bank 'CBE' not found in bank_ids
Skipping row 8: Bank 'CBE' not found in bank_ids
Skipping row 9: Bank 'CBE' not found in bank_ids
Skipping row 10: Bank 'CBE' not found in bank_ids
Skipping row 11: Bank 'CBE' not found in bank_ids
Skipping row 12: Bank 'CBE' not found in bank_ids
Skipping row 13: Bank 'CBE' not found in bank_ids
Skipping row 14: Bank 'CBE' not found in bank_ids
Skipping row 15: Bank 'CBE' not found in bank_ids
Skipping row 16: Bank 'CBE' not found in bank_ids
Skipping row 17: Bank 'CBE' not found in bank_ids
Skipping row 18: Bank 'CBE' not found in bank_ids
Skipping row 19: Bank 'CBE' not found in bank_ids
Skipping r

In [33]:
try:
    conn.commit()
    print("All data inserted successfully.")
except cx_Oracle.DatabaseError as e:
    print(f"Error during commit: {e}")
finally:
    cursor.close()
    conn.close()

All data inserted successfully.
